In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/events.csv', low_memory=False)

In [3]:
def asignar_empresa(celular):
    celular = celular.lower()
    if ('samsung' in celular):
        return 'Samsung'
    if ('iphone' in celular or 'ipad' in celular):
        return 'Apple'
    if ('sony' in celular):
        return 'Sony'
    if ('motorola' in celular):
        return 'Motorola'
    if ('asus' in celular):
        return 'Asus'
    if ('lg' in celular):
        return 'LG'
    if ('lenovo' in celular):
        return 'Lenovo'
    if ('quantum' in celular):
        return 'Quantum'
    return 'Otros'

In [4]:
modelos_por_empresa = df.loc[df['sku'].notnull(), :].dropna(axis=1).copy()
modelos_por_empresa['empresa'] = modelos_por_empresa['model'].apply(lambda x: asignar_empresa(x))

## Marca más vista por usuario

In [5]:
vistos = modelos_por_empresa.loc[modelos_por_empresa['event'] == 'viewed product']

In [6]:
vistos = vistos.groupby('person')['empresa'].value_counts()
vistos = vistos.reset_index(level='empresa', name='count')
vistos = vistos[~vistos.index.duplicated(keep='first')][['empresa']]

In [7]:
vistos = pd.get_dummies(vistos['empresa'], prefix='vio_mas', dtype=bool)

In [8]:
vistos.tail()

,vio_mas_Apple,vio_mas_Asus,vio_mas_LG,vio_mas_Lenovo,vio_mas_Motorola,vio_mas_Quantum,vio_mas_Samsung,vio_mas_Sony
person,,,,,,,,
fff1caee,False,False,True,False,False,False,False,False
fff2bdde,False,False,False,False,False,False,True,False
fff72025,False,False,True,False,False,False,False,False
fff78145,True,False,False,False,False,False,False,False
fffd1246,True,False,False,False,False,False,False,False


In [9]:
users = pd.read_csv('data/features_basicas.csv', low_memory=False, index_col='person')
users.shape

(38829, 6)

In [10]:
usuarios_con_productos_vistos = users.join(vistos)

In [11]:
usuarios_con_productos_vistos = usuarios_con_productos_vistos.fillna({'vio_mas_Apple':False, 'vio_mas_Samsung':False, 'vio_mas_Lenovo':False,\
                      'vio_mas_LG':False, 'vio_mas_Asus':False, 'vio_mas_Motorola':False,\
                      'vio_mas_Quantum':False, 'vio_mas_Sony':False})
usuarios_con_productos_vistos.columns

Index(['vio_productos', 'tiene_checkouts', 'tiene_conversions',
       'cant_prod_vistos', 'cantidad_checkouts', 'cantidad_conversions',
       'vio_mas_Apple', 'vio_mas_Asus', 'vio_mas_LG', 'vio_mas_Lenovo',
       'vio_mas_Motorola', 'vio_mas_Quantum', 'vio_mas_Samsung',
       'vio_mas_Sony'],
      dtype='object')

In [12]:
# Descomentar para guardar, un horror
#usuarios_con_productos_vistos[['vio_mas_Apple',
# 'vio_mas_Asus',
# 'vio_mas_LG',
# 'vio_mas_Lenovo',
# 'vio_mas_Motorola',
# 'vio_mas_Quantum',
# 'vio_mas_Samsung',
# 'vio_mas_Sony']].to_csv('data/features_vistos.csv', sep=',')

## Compra según marca

In [13]:
compras = modelos_por_empresa.loc[modelos_por_empresa['event'] == 'conversion']

In [14]:
compras_agrupadas = compras.groupby('person')['empresa'].value_counts().unstack(fill_value=0)

In [15]:
compras_agrupadas = compras_agrupadas.apply(lambda x: x > 0, axis=1)
compras_agrupadas = compras_agrupadas.add_prefix('compro_')
compras_agrupadas.shape

(4293, 8)

In [16]:
users_con_compras = users.join(compras_agrupadas)

In [17]:
users_con_compras.shape

(38829, 14)

In [18]:
users_con_compras.fillna(False, inplace=True)

In [19]:
users_con_compras.columns

Index(['vio_productos', 'tiene_checkouts', 'tiene_conversions',
       'cant_prod_vistos', 'cantidad_checkouts', 'cantidad_conversions',
       'compro_Apple', 'compro_Asus', 'compro_LG', 'compro_Lenovo',
       'compro_Motorola', 'compro_Quantum', 'compro_Samsung', 'compro_Sony'],
      dtype='object')

In [20]:
# Descomentar para guardar, un horror
#users_con_compras[['compro_Apple', 'compro_Asus', 'compro_LG', 'compro_Lenovo',
#       'compro_Motorola', 'compro_Quantum', 'compro_Samsung', 'compro_Sony']].to_csv('data/features_compras.csv')